In [5]:
# 1. Imports
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
import itertools

# 2. Load processed data
train = pd.read_csv("processed_data/train.csv")
val = pd.read_csv("processed_data/val.csv")
test = pd.read_csv("processed_data/test.csv")

X_train = train.drop("G3_binary", axis=1)
y_train = train["G3_binary"]
X_val = val.drop("G3_binary", axis=1)
y_val = val["G3_binary"]
X_test = test.drop("G3_binary", axis=1)
y_test = test["G3_binary"]

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {0: class_weights[0], 1: class_weights[1]}

# 3. Hyperparameter grid (16 combos)
hidden_layers_options = [
    [10],
    [16,8],
    [32,16],
    [64,32]
]
learning_rates = [0.001, 0.01]
dropout_rates = [0.0, 0.3]
output_activation = ['sigmoid']  # keep fixed
batch_size = 32

# 4. Function to build and train
def build_and_train(layers_config, lr, dropout, run_num):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train.shape[1],)))
    for units in layers_config:
        model.add(layers.Dense(units, activation='relu'))
        if dropout > 0:
            model.add(layers.Dropout(dropout))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss="binary_crossentropy",
                  metrics=["accuracy", keras.metrics.AUC(name="auc")])
    
    model.fit(X_train, y_train,
              validation_data=(X_val, y_val),
              epochs=50,
              batch_size=batch_size,
              class_weight=class_weights_dict,
              verbose=0)
    
    probs = model.predict(X_test).ravel()
    preds = (probs > 0.5).astype(int)
    
    return {
        "Model No.": run_num,
        "Layers": str(layers_config),
        "Learning Rate": lr,
        "Dropout": dropout,
        "Accuracy": accuracy_score(y_test, preds),
        "Precision": precision_score(y_test, preds, zero_division=0),
        "Recall": recall_score(y_test, preds, zero_division=0),
        "F1-Score": f1_score(y_test, preds, zero_division=0),
        "ROC-AUC": roc_auc_score(y_test, probs)
    }

# 5. Run models
results = []
run_num = 1
for layers_config, lr, dr in itertools.product(hidden_layers_options, learning_rates, dropout_rates):
    print(f"Running Model {run_num}: Layers={layers_config}, LR={lr}, Dropout={dr}")
    res = build_and_train(layers_config, lr, dr, run_num)
    results.append(res)
    run_num += 1

# 6. Results table
results_df = pd.DataFrame(results)
display(results_df)

# Save to CSV
results_df.to_csv("hyperparameter_results.csv", index=False)
print("✅ Results saved to hyperparameter_results.csv")


Running Model 1: Layers=[10], LR=0.001, Dropout=0.0
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Running Model 2: Layers=[10], LR=0.001, Dropout=0.3
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Running Model 3: Layers=[10], LR=0.01, Dropout=0.0
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Running Model 4: Layers=[10], LR=0.01, Dropout=0.3
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Running Model 5: Layers=[16, 8], LR=0.001, Dropout=0.0
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Running Model 6: Layers=[16, 8], LR=0.001, Dropout=0.3
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Running Model 7: Layers=[16, 8], LR=0.01, Dropout=0.0
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Running Model 8: Layers=[16, 8], LR=0.01, Dropout=0.3
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Running Model 9: Layers=[32, 16], LR=0.001, Dropout=0.0
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Running Model 10: Layers=[32, 16], LR=0.001, Dropout=0.3
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Running Model 11: Layers=[32, 16], LR=0.01, Dropout=0.0
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s

,Model No.,Layers,Learning Rate,Dropout,Accuracy,Precision,Recall,F1-Score,ROC-AUC
0,1,[10],0.001,0.0,0.650000,0.272727,0.545455,0.363636,0.699443
1,2,[10],0.001,0.3,0.750000,0.388889,0.636364,0.482759,0.727273
2,3,[10],0.010,0.0,0.833333,0.555556,0.454545,0.500000,0.669759
3,4,[10],0.010,0.3,0.666667,0.235294,0.363636,0.285714,0.679035
4,5,"[16, 8]",0.001,0.0,0.716667,0.312500,0.454545,0.370370,0.632653
5,6,"[16, 8]",0.001,0.3,0.683333,0.318182,0.636364,0.424242,0.686456
6,7,"[16, 8]",0.010,0.0,0.800000,0.400000,0.181818,0.250000,0.582560
7,8,"[16, 8]",0.010,0.3,0.700000,0.266667,0.363636,0.307692,0.660482
8,9,"[32, 16]",0.001,0.0,0.766667,0.363636,0.363636,0.363636,0.717996
9,10,"[32, 16]",0.001,0.3,0.750000,0.388889,0.636364,0.482759,0.760668


✅ Results saved to hyperparameter_results.csv
